In [65]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
%matplotlib inline
import seaborn as sns; sns.set_theme(color_codes=True)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
import math

In [66]:
city = 'oradell'

In [97]:
#read in labels
labels = pd.read_csv('../../../data/' + city + '/processed-labels/labels_raw_' + city + '.csv')
labels

,audit_task_id,label_id,gsv_panorama_id_x,label_type,severity,correct,high_quality_user,geometry,gsv_panorama_id_y,zoom,heading,pitch,photographer_heading,photographer_pitch,user_id,lat,lng
0,7,9,8AsY4nlOJQAo8P5YYaEjzA,CurbRamp,1.0,True,True,POINT (-74.03411102294922 40.945518493652344),8AsY4nlOJQAo8P5YYaEjzA,2,309.035706,-15.723214,108.536644,-0.496017,503d2482-420f-454e-8118-aac2d22592aa,40.945518,-74.034111
1,7,10,8AsY4nlOJQAo8P5YYaEjzA,NoSidewalk,5.0,True,True,POINT (-74.03414154052734 40.94554138183594),8AsY4nlOJQAo8P5YYaEjzA,2,303.232147,-13.580358,108.536644,-0.496017,503d2482-420f-454e-8118-aac2d22592aa,40.945541,-74.034142
2,7,11,8AsY4nlOJQAo8P5YYaEjzA,CurbRamp,1.0,True,True,POINT (-74.0341567993164 40.9454345703125),8AsY4nlOJQAo8P5YYaEjzA,2,266.714294,-12.330358,108.536644,-0.496017,503d2482-420f-454e-8118-aac2d22592aa,40.945435,-74.034157
3,7,12,8AsY4nlOJQAo8P5YYaEjzA,NoSidewalk,5.0,True,True,POINT (-74.0342025756836 40.9454460144043),8AsY4nlOJQAo8P5YYaEjzA,2,270.821442,-11.616072,108.536644,-0.496017,503d2482-420f-454e-8118-aac2d22592aa,40.945446,-74.034203
4,7,13,8AsY4nlOJQAo8P5YYaEjzA,Crosswalk,1.0,True,True,POINT (-74.03414154052734 40.945472717285156),8AsY4nlOJQAo8P5YYaEjzA,2,284.392853,-13.758928,108.536644,-0.496017,503d2482-420f-454e-8118-aac2d22592aa,40.945473,-74.034142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12129,2194,13096,28tGbhCsMyFUsKRs7-jp_A,NoCurbRamp,2.0,True,True,POINT (-74.03062438964844 40.954376220703125),28tGbhCsMyFUsKRs7-jp_A,2,189.401779,-10.892858,287.290100,-2.176155,528eb1d6-e20e-4fb9-81c9-0c06b9e2852f,40.954376,-74.030624
12130,2194,13097,28tGbhCsMyFUsKRs7-jp_A,Obstacle,4.0,NaN,True,POINT (-74.0306167602539 40.95439147949219),28tGbhCsMyFUsKRs7-jp_A,2,189.401779,-10.892858,287.290100,-2.176155,528eb1d6-e20e-4fb9-81c9-0c06b9e2852f,40.954391,-74.030617
12131,2194,13098,5zDLTjYdw6hQhfsXppE0cA,SurfaceProblem,1.0,True,True,POINT (-74.03089141845703 40.95462417602539),5zDLTjYdw6hQhfsXppE0cA,2,77.437500,-23.928572,287.278870,-2.803400,528eb1d6-e20e-4fb9-81c9-0c06b9e2852f,40.954624,-74.030891
12132,2202,13099,WkdNrR64GZUC1V_at1pxyA,NoCurbRamp,NaN,NaN,False,POINT (-74.02997589111328 40.96918869018555),WkdNrR64GZUC1V_at1pxyA,1,178.687500,-8.625000,281.559601,-2.859410,0cfe6980-35ac-4cdd-a2df-cbee02531406,40.969189,-74.029976


In [98]:
#drop columns audit_task_id, correct, high_quality_user, gsv_panorama_id_y
labels = labels.drop(columns=['audit_task_id', 'correct', 'high_quality_user', 'gsv_panorama_id_y'])
#renname gsv_panorama_id_y to gsv_panorama_id
labels = labels.rename(columns={'gsv_panorama_id_x': 'gsv_panorama_id'})

In [99]:
#read in all files in the feature folder
clustered = pd.read_csv('../../../data/' + city + '/features/clustered.csv')
distance_to_road = pd.read_csv('../../../data/' + city + '/features/distance_to_road.csv')
distance_to_intersection = pd.read_csv('../../../data/' + city + '/features/distance_to_intersection.csv')
optional_input = pd.read_csv('../../../data/' + city + '/features/optional_input.csv')


In [100]:
#merge all into one on label_id
df = labels.merge(clustered, on='label_id').merge(distance_to_road, on='label_id').merge(distance_to_intersection, on='label_id').merge(optional_input, on='label_id')

In [101]:
#keep only the label types Curb Ramp, Missing Curb Ramp, Obstacle in Path, Surface Problem, and No Sidewalk
df = df[df['label_type'].isin(['CurbRamp', 'NoCurbRamp', 'Obstacle', 'SurfaceProblem', 'NoSidewalk'])]

In [102]:
#drop nan in severity
df = df.dropna(subset=['severity'])

In [103]:
df

,label_id,gsv_panorama_id,label_type,severity,geometry,zoom,heading,pitch,photographer_heading,photographer_pitch,user_id,lat,lng,cluster_id,clustered,cluster_label_count,way_type,distance_to_road,distance_to_intersection,tag,tag_count,description
0,9,8AsY4nlOJQAo8P5YYaEjzA,CurbRamp,1.0,POINT (-74.03411102294922 40.945518493652344),2,309.035706,-15.723214,108.536644,-0.496017,503d2482-420f-454e-8118-aac2d22592aa,40.945518,-74.034111,1205,1,2,residential,8.719586,16.196217,0.0,0.0,0.0
1,10,8AsY4nlOJQAo8P5YYaEjzA,NoSidewalk,5.0,POINT (-74.03414154052734 40.94554138183594),2,303.232147,-13.580358,108.536644,-0.496017,503d2482-420f-454e-8118-aac2d22592aa,40.945541,-74.034142,3290,0,1,residential,13.715870,0.000000,1.0,1.0,0.0
2,11,8AsY4nlOJQAo8P5YYaEjzA,CurbRamp,1.0,POINT (-74.0341567993164 40.9454345703125),2,266.714294,-12.330358,108.536644,-0.496017,503d2482-420f-454e-8118-aac2d22592aa,40.945435,-74.034157,1237,1,2,residential,16.168285,28.692549,0.0,0.0,0.0
3,12,8AsY4nlOJQAo8P5YYaEjzA,NoSidewalk,5.0,POINT (-74.0342025756836 40.9454460144043),2,270.821442,-11.616072,108.536644,-0.496017,503d2482-420f-454e-8118-aac2d22592aa,40.945446,-74.034203,3293,0,1,residential,24.682769,0.000000,1.0,1.0,0.0
5,14,8AsY4nlOJQAo8P5YYaEjzA,CurbRamp,1.0,POINT (-74.03398132324219 40.94548034667969),1,45.428570,-33.785713,108.536644,-0.496017,503d2482-420f-454e-8118-aac2d22592aa,40.945480,-74.033981,1183,1,2,residential,7.900726,25.954706,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12126,13093,28tGbhCsMyFUsKRs7-jp_A,CurbRamp,2.0,POINT (-74.03059387207031 40.95452117919922),2,358.151794,-35.000000,287.290100,-2.176155,528eb1d6-e20e-4fb9-81c9-0c06b9e2852f,40.954521,-74.030594,525,1,2,residential,7.291089,165.744075,1.0,2.0,0.0
12128,13095,28tGbhCsMyFUsKRs7-jp_A,CurbRamp,2.0,POINT (-74.0306167602539 40.95445251464844),2,189.937500,-19.107143,287.290100,-2.176155,528eb1d6-e20e-4fb9-81c9-0c06b9e2852f,40.954453,-74.030617,523,1,2,residential,33.074767,143.102128,1.0,2.0,0.0
12129,13096,28tGbhCsMyFUsKRs7-jp_A,NoCurbRamp,2.0,POINT (-74.03062438964844 40.954376220703125),2,189.401779,-10.892858,287.290100,-2.176155,528eb1d6-e20e-4fb9-81c9-0c06b9e2852f,40.954376,-74.030624,2045,0,1,residential,60.339565,117.545556,1.0,1.0,0.0
12130,13097,28tGbhCsMyFUsKRs7-jp_A,Obstacle,4.0,POINT (-74.0306167602539 40.95439147949219),2,189.401779,-10.892858,287.290100,-2.176155,528eb1d6-e20e-4fb9-81c9-0c06b9e2852f,40.954391,-74.030617,7926,0,1,residential,54.412181,0.000000,0.0,0.0,1.0


In [104]:
gt = pd.read_csv('../../../data/' + city + '/features/' + city + '_ground_truth_labels.csv')

In [105]:
gt

,label_id,ground_truth
0,9,1
1,10,1
2,11,1
3,12,1
4,13,1
...,...,...
4138,11753,1
4139,12092,1
4140,12539,1
4141,12837,1


In [106]:
#merge gt with labels how left
test= pd.merge(df, gt, on='label_id', how='inner')

In [107]:
#rename ground truth to verified
test.rename(columns={'ground_truth': 'verified'}, inplace=True)

In [108]:
test.verified.value_counts()

1    3304
0     271
Name: verified, dtype: int64

In [109]:
#if the label_id in gt take it out of df
training = df[~df.label_id.isin(test.label_id)]

In [110]:
training

,label_id,gsv_panorama_id,label_type,severity,geometry,zoom,heading,pitch,photographer_heading,photographer_pitch,user_id,lat,lng,cluster_id,clustered,cluster_label_count,way_type,distance_to_road,distance_to_intersection,tag,tag_count,description
48,59,7l-j4D7uhfKRQPpYXIU59w,Obstacle,2.0,POINT (-74.03707122802734 40.94615936279297),3,225.573654,-11.361608,20.765854,0.262733,503d2482-420f-454e-8118-aac2d22592aa,40.946159,-74.037071,8057,0,1,residential,10.535272,0.000000,1.0,1.0,0.0
117,137,adXdsY189n44f0_ae5Hptg,NoCurbRamp,5.0,POINT (-74.03673553466797 40.96704864501953),1,278.325348,-27.120001,291.724182,3.633377,e86fdc72-c79d-4cdc-b5b2-8ef4859141d2,40.967049,-74.036736,2573,0,1,residential,1.626180,35.370199,1.0,1.0,0.0
119,139,adXdsY189n44f0_ae5Hptg,SurfaceProblem,1.0,POINT (-74.03650665283203 40.96693420410156),2,158.084290,-17.280714,291.724182,3.633377,e86fdc72-c79d-4cdc-b5b2-8ef4859141d2,40.966934,-74.036507,8598,1,2,residential,21.374182,0.000000,1.0,2.0,0.0
122,142,adXdsY189n44f0_ae5Hptg,SurfaceProblem,1.0,POINT (-74.03662109375 40.96712112426758),2,324.155701,-22.816429,291.724182,3.633377,e86fdc72-c79d-4cdc-b5b2-8ef4859141d2,40.967121,-74.036621,11265,0,1,residential,33.876173,0.000000,1.0,2.0,0.0
132,149,ZPQwHRwEFqkXNlzq_VuPZQ,SurfaceProblem,2.0,POINT (-74.036865234375 40.967185974121094),3,312.260620,-14.562500,287.279633,2.883698,e86fdc72-c79d-4cdc-b5b2-8ef4859141d2,40.967186,-74.036865,11268,0,1,residential,32.578955,0.000000,1.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12126,13093,28tGbhCsMyFUsKRs7-jp_A,CurbRamp,2.0,POINT (-74.03059387207031 40.95452117919922),2,358.151794,-35.000000,287.290100,-2.176155,528eb1d6-e20e-4fb9-81c9-0c06b9e2852f,40.954521,-74.030594,525,1,2,residential,7.291089,165.744075,1.0,2.0,0.0
12128,13095,28tGbhCsMyFUsKRs7-jp_A,CurbRamp,2.0,POINT (-74.0306167602539 40.95445251464844),2,189.937500,-19.107143,287.290100,-2.176155,528eb1d6-e20e-4fb9-81c9-0c06b9e2852f,40.954453,-74.030617,523,1,2,residential,33.074767,143.102128,1.0,2.0,0.0
12129,13096,28tGbhCsMyFUsKRs7-jp_A,NoCurbRamp,2.0,POINT (-74.03062438964844 40.954376220703125),2,189.401779,-10.892858,287.290100,-2.176155,528eb1d6-e20e-4fb9-81c9-0c06b9e2852f,40.954376,-74.030624,2045,0,1,residential,60.339565,117.545556,1.0,1.0,0.0
12130,13097,28tGbhCsMyFUsKRs7-jp_A,Obstacle,4.0,POINT (-74.0306167602539 40.95439147949219),2,189.401779,-10.892858,287.290100,-2.176155,528eb1d6-e20e-4fb9-81c9-0c06b9e2852f,40.954391,-74.030617,7926,0,1,residential,54.412181,0.000000,0.0,0.0,1.0


In [111]:
test

,label_id,gsv_panorama_id,label_type,severity,geometry,zoom,heading,pitch,photographer_heading,photographer_pitch,user_id,lat,lng,cluster_id,clustered,cluster_label_count,way_type,distance_to_road,distance_to_intersection,tag,tag_count,description,verified
0,9,8AsY4nlOJQAo8P5YYaEjzA,CurbRamp,1.0,POINT (-74.03411102294922 40.945518493652344),2,309.035706,-15.723214,108.536644,-0.496017,503d2482-420f-454e-8118-aac2d22592aa,40.945518,-74.034111,1205,1,2,residential,8.719586,16.196217,0.0,0.0,0.0,1
1,10,8AsY4nlOJQAo8P5YYaEjzA,NoSidewalk,5.0,POINT (-74.03414154052734 40.94554138183594),2,303.232147,-13.580358,108.536644,-0.496017,503d2482-420f-454e-8118-aac2d22592aa,40.945541,-74.034142,3290,0,1,residential,13.715870,0.000000,1.0,1.0,0.0,1
2,11,8AsY4nlOJQAo8P5YYaEjzA,CurbRamp,1.0,POINT (-74.0341567993164 40.9454345703125),2,266.714294,-12.330358,108.536644,-0.496017,503d2482-420f-454e-8118-aac2d22592aa,40.945435,-74.034157,1237,1,2,residential,16.168285,28.692549,0.0,0.0,0.0,1
3,12,8AsY4nlOJQAo8P5YYaEjzA,NoSidewalk,5.0,POINT (-74.0342025756836 40.9454460144043),2,270.821442,-11.616072,108.536644,-0.496017,503d2482-420f-454e-8118-aac2d22592aa,40.945446,-74.034203,3293,0,1,residential,24.682769,0.000000,1.0,1.0,0.0,1
4,14,8AsY4nlOJQAo8P5YYaEjzA,CurbRamp,1.0,POINT (-74.03398132324219 40.94548034667969),1,45.428570,-33.785713,108.536644,-0.496017,503d2482-420f-454e-8118-aac2d22592aa,40.945480,-74.033981,1183,1,2,residential,7.900726,25.954706,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3570,11752,SNQ1WaQPKcAkvTOTAtULJA,Obstacle,2.0,POINT (-74.0453109741211 40.96165084838867),2,137.830353,-11.071428,108.567726,0.086418,2d7e45f5-3656-4aae-9a09-14addfa80d22,40.961651,-74.045311,8007,1,2,residential,22.926788,0.000000,1.0,1.0,0.0,1
3571,11753,SNQ1WaQPKcAkvTOTAtULJA,Obstacle,2.0,POINT (-74.0452880859375 40.96165466308594),2,137.830353,-11.071428,108.567726,0.086418,2d7e45f5-3656-4aae-9a09-14addfa80d22,40.961655,-74.045288,8006,0,1,residential,19.500529,0.000000,1.0,1.0,0.0,1
3572,12092,F_sesLbDcaUyjG7w_nwdzg,Obstacle,4.0,POINT (-74.03170013427734 40.94500732421875),3,36.669643,-8.265625,19.918083,1.928528,e86fdc72-c79d-4cdc-b5b2-8ef4859141d2,40.945007,-74.031700,7996,0,1,residential,21.555111,0.000000,1.0,1.0,0.0,1
3573,12539,QSi-X0zwaqMR5rhs09kYig,CurbRamp,1.0,POINT (-74.01839447021484 40.95423889160156),1,90.812500,-14.000000,338.763458,0.073710,52b661f5-2ecd-4733-acc4-dad4b32e319e,40.954239,-74.018394,594,1,4,residential,11.219803,30.938699,0.0,0.0,0.0,1


In [112]:
training.to_csv('../../../data/' + city + '/processed-labels/training_set_' + city + '.csv', index=False)
test.to_csv('../../../data/' + city + '/processed-labels/test_set_' + city + '.csv', index=False)
df.to_csv('../../../data/' + city + '/processed-labels/labels_full_features_' + city + '.csv', index=False)